*tool - jupyter notebook 

## 마케팅 효과 분석: 마케팅 포인트와 주가와의 상관관계 분석

## 1. 주가정보 획득
* 마케팅 포인트와 주가와의 상관관계 분석을 위한 주가 획득


* 예시)
    * 미디어주 분석
    * CJ ENM(035760)  
         - https://finance.naver.com/item/sise_day.nhn?code=069960&page=1

### 0) 패키지 설치

In [2]:
import sqlite3
import requests # 내부 source 를 가져오는 라이브러리
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### 1) 주가 정보 크롤링

- 함수: stockGet(url='')

In [57]:
# 함수 선언부 url을 매개변수화
def stockGet(url=''):
    src = requests.get(url)
    result = src.content.strip()
    html = BeautifulSoup(result,'html.parser')
    tgts = html.select('table.type2 tr')
    
    res = []
    for tgt in tgts:
        if(tgt.select('td .p10')): 
            sday = tgt.select('td .p10')[0].text.replace(".","")
            
            endp = tgt.select('td.num')[0].text.replace(",","")
            startp = tgt.select('td.num')[2].text.replace(",","")
            maxp = tgt.select('td.num')[3].text.replace(",","")
            minp = tgt.select('td.num')[4].text.replace(",","")
            vol = tgt.select('td.num')[5].text.replace(",","")
            res.append((sday,startp,maxp,minp,endp,vol))

    return res

- 함수화 과정
    + [select 참고](https://hogni.tistory.com/46)

In [39]:
url = 'https://finance.naver.com/item/sise_day.nhn?code=069960&page=1'
src = requests.get(url)
result = src.content.strip() # strip(char): 문자열에서 특정 문자(char) 제거
html = BeautifulSoup(result,'html.parser') # 구문 분석
tgts = html.select('table.type2 tr') # select 태그찾기: 자손(공백), 자식(>)

# len(tgts) = 16개 > 필요없는 tr이 존재함 > 정제 필요

[<span class="tah p10 gray03">2020.09.16</span>]

In [43]:
tgts[3].select('.tah.p10.gray03')[0] 
tgts[3].select('td .p10')[0] 
# 날짜가 들어가 있는 tr에는 모두 (td 자식 중) p10라는 클래스 요소가 있음

<span class="tah p10 gray03">2020.09.16</span>

In [49]:
(tgts[3].select('td.num')[2].text, tgts[3].select('td.num')[2].text)

('57,600', '57,600')

In [52]:
res = []
for tgt in tgts:
    if(tgt.select('td .p10')): # 날짜있는 줄만 취급
        sday = tgt.select('td .p10')[0].text.replace(".","")
        # 날짜 가져오기
        endp = tgt.select('td.num')[0].text.replace(",","")
        startp = tgt.select('td.num')[2].text.replace(",","")
        maxp = tgt.select('td.num')[3].text.replace(",","")
        minp = tgt.select('td.num')[4].text.replace(",","")
        vol = tgt.select('td.num')[5].text.replace(",","")
        res.append((sday,startp,maxp,minp,endp,vol))
res

[('20200917', '57700', '57700', '56500', '57000', '220136'),
 ('20200916', '57600', '58500', '57100', '57700', '257684'),
 ('20200915', '57500', '58000', '56600', '57400', '219583'),
 ('20200914', '57000', '58400', '56700', '57200', '324102'),
 ('20200911', '56400', '57400', '56200', '56300', '269437'),
 ('20200910', '56500', '57200', '55900', '56400', '262036'),
 ('20200909', '55900', '56600', '55400', '56000', '125147'),
 ('20200908', '56800', '57100', '56300', '56400', '152605'),
 ('20200907', '56100', '57100', '55600', '56300', '174472'),
 ('20200904', '55300', '56200', '55100', '55800', '194702')]

### 2) SQLite 에 데이터 입력

- 사전 준비
    + DB Browser for SQLite [다운](https://sqlitebrowser.org/dl/)
    + 'NaverStock.db' 데이터 베이스 만들기
    + 'StockDay' 테이블 추가하기
    + 컬럼 추가하기: SIDX,SCODE,SCOMP,SDAY,OPENP,HIGHP,LOWP,CLOSEP,VOL)
    + 데이터 타입: (SCODE,SCOMP-TEXT),(SDAY-INT),(나머지-NUMERIC)
    
    
- 함수 
    + addData(data): 데이터 추가 함수
    + selData(data): 데이터 선택 함수 - 데이터 추가 시 중복방지용

In [54]:
def addData(data):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql = 'insert into StockDay \
        (SIDX,SCODE,SCOMP,SDAY,OPENP,HIGHP,LOWP,CLOSEP,VOL) \
        values (?,?,?,?,?,?,?,?,?)'
        ##### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
        cur.execute(sql,data)
        conn.commit()
        cur.close()

In [55]:
dbname='NaverStock.db'
def selData(whr=' where SIDX=0'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql='select SIDX from StockDay '+whr
        cur.execute(sql)
        rows=cur.fetchall()
        cur.close()
    return rows

### 3) 최종: 주가 정보 크롤링 + 입력

- 함수: MyCrawl(comp=('현대백화점','069960'),sp=int,ep=int)

In [63]:
def MyCrawl(comp=('현대백화점','069960'),sp=int,ep=int):
    sCode = comp[1]
    sComp = comp[0]
    urlsrc = 'https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}'
    for i in range(sp,ep):
        print('#'*30)
        url = urlsrc.format(sCode, i)
        print(url)
        # 데이터 크롤링(stockGet)
        rs = stockGet(url)

        for r in rs:
            sidx = r[0]+sCode
            data = [sidx, sCode, sComp, r[0], r[1], r[2], r[3], r[4], r[5]]
            # 데이터 입력: 중복데이터 확인 > 입력
            preData = selData(' where SIDX='+sidx)
            if(preData):
                print("중복데이터입니다.", preData)
            else:
                addData(data)

- 함수화 과정

In [59]:
urlsrc = 'https://finance.naver.com/item/sise_day.nhn?code={0}&page={1}'
sCode = '069960'
sComp = '현대백화점'
for i in range(1,2):
    print('#'*30)
    url = urlsrc.format(sCode, i)
    print(url)
    # 데이터 크롤링(stockGet)
    rs = stockGet(url)
    
    for r in rs:
        sidx = r[0]+sCode
        ### data=[1001,60000,' 신세계','20200830',30,60,70,50,250]
        data = [sidx, sCode, sComp, r[0], r[1], r[2], r[3], r[4], r[5]]
        # 데이터 입력: 중복데이터 확인 > 입력
        preData = selData(' where SIDX='+sidx)
        if(preData):
            print("중복데이터입니다.", preData)
        else:
            # print(r)
            addData(data)

##############################
https://finance.naver.com/item/sise_day.nhn?code=069960&page=1
중복데이터입니다. [(20200917069960,)]
중복데이터입니다. [(20200916069960,)]
중복데이터입니다. [(20200915069960,)]
중복데이터입니다. [(20200914069960,)]
중복데이터입니다. [(20200911069960,)]
중복데이터입니다. [(20200910069960,)]
중복데이터입니다. [(20200909069960,)]
중복데이터입니다. [(20200908069960,)]
중복데이터입니다. [(20200907069960,)]
중복데이터입니다. [(20200904069960,)]


### 4) 최종 함수 실습하기

In [66]:
compList = [('CJENM','035760'),('GS홈쇼핑','028150')]

In [67]:
MyCrawl(compList[0],1,5)

##############################
https://finance.naver.com/item/sise_day.nhn?code=035760&page=1
중복데이터입니다. [(20200916035760,)]
중복데이터입니다. [(20200915035760,)]
중복데이터입니다. [(20200914035760,)]
중복데이터입니다. [(20200911035760,)]
중복데이터입니다. [(20200910035760,)]
중복데이터입니다. [(20200909035760,)]
중복데이터입니다. [(20200908035760,)]
중복데이터입니다. [(20200907035760,)]
중복데이터입니다. [(20200904035760,)]
##############################
https://finance.naver.com/item/sise_day.nhn?code=035760&page=2
중복데이터입니다. [(20200903035760,)]
중복데이터입니다. [(20200902035760,)]
중복데이터입니다. [(20200901035760,)]
중복데이터입니다. [(20200831035760,)]
중복데이터입니다. [(20200828035760,)]
중복데이터입니다. [(20200827035760,)]
중복데이터입니다. [(20200826035760,)]
중복데이터입니다. [(20200825035760,)]
중복데이터입니다. [(20200824035760,)]
중복데이터입니다. [(20200821035760,)]
##############################
https://finance.naver.com/item/sise_day.nhn?code=035760&page=3
중복데이터입니다. [(20200820035760,)]
중복데이터입니다. [(20200819035760,)]
중복데이터입니다. [(20200818035760,)]
중복데이터입니다. [(20200814035760,)]
중복데이터입니다. [(20200813035760,)